In [1]:
%%capture
!pip3 install --upgrade pip numpy pytorch-lightning albumentations==0.4.6

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import math
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, random_split
import torchvision
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from torchvision import datasets, transforms, models

import albumentations as A
from albumentations.pytorch import ToTensorV2

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
import pandas

In [51]:
def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            coords = np.array([float(x) for x in parts[1:]], dtype='float64')
            res[parts[0]] = coords
    return res
y = read_csv('gt.csv')
torch.from_numpy(y['0000.jpg'])

tensor([0.], dtype=torch.float64)

In [5]:
def get_image(path, transform = None):    
    
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (100, 100))
    return img

In [60]:
class Dataset(Dataset):
  def __init__(self, mode, data_dir, labels, fraction, transform = None):
    self.transform = transform
    self.items = []

    pathes = os.listdir(data_dir)
    pathes.sort()
    if mode == 'train':
      img_pathes = pathes[int(len(pathes) * fraction):]
    elif mode == 'test':
      img_pathes = pathes[:100] #[:int(len(pathes)*fraction)]

    for path in img_pathes:
      self.items.append((
          os.path.join(data_dir, path),
          int(labels[path][0])
      ))

  def __len__(self):
    return len(self.items)

  def __getitem__(self, index):

    path, label = self.items[index]
    img = get_image(path)
    
    if self.transform:
      transformed = self.transform(image = img)

    img = img.transpose(2,0,1)
    
    return torch.tensor(img, dtype = torch.float), torch.tensor(label)

In [61]:
train_ds = Dataset(mode = 'train', data_dir = '/content/drive/MyDrive/00_test_img_input/train/images', labels=y, fraction=0.8)
val_ds = Dataset(mode = 'test', data_dir = '/content/drive/MyDrive/00_test_img_input/train/images', labels=y, fraction=0.2)

train_dl = DataLoader(train_ds, batch_size=5, shuffle = True)
val_dl = DataLoader(val_ds, batch_size=5, shuffle=False)

In [73]:
class myModel(pl.LightningModule):
  def __init__(self, num_classes, lr_rate):
        super().__init__() 

        self.lr_rate = lr_rate
        self.model = mobilenet_v2(weights = MobileNet_V2_Weights.IMAGENET1K_V2)

        Linear_size = list(self.model.children())[-1][-1].out_features
        self.output = nn.Linear(1000, num_classes)
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax()

        for child in list(self.model.children())[:-6]:
                for param in child.parameters():
                    param.requires_grad = True
        
        self.loss = F.cross_entropy
        
  def forward(self, x):
      x = self.model(x)
      x = self.dropout(x)
      x = self.softmax(self.output(x))
      return x

  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = self.loss(logits, y)
      logs = {'train_loss': loss}
      acc = torch.sum(logits.argmax(dim=1) == y) / y.shape[0]
      self.log('val_loss', loss, on_step=True, on_epoch=False)
      self.log('val_acc', acc, on_step=True, on_epoch=False)
      return {'val_loss': loss, 'val_acc': acc}

  def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      logits = self.forward(x)
      loss = self.loss(logits, y)
      acc = torch.sum(logits.argmax(dim=1) == y) / y.shape[0]
  
      return {'loss': loss, 'acc':acc}

  def validation_epoch_end(self, outputs):
      avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
      avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
      self.log('val_acc', avg_acc, on_epoch=True, on_step=False)
      print(f"Val_loss: {avg_loss:.2f}, Val_acc: {avg_acc:.2f}", end = " ")
  
  def training_epoch_end(self, outputs):
      avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
      avg_acc = torch.stack([x['acc'] for x in outputs]).mean()
      #self.log('train_acc', avg_acc, on_epoch=True, on_step=False)
      print(f"Train_loss: {avg_loss:.2f}, train_acc: {avg_acc:.2f}", end = " ")

  def configure_optimizers(self):

    optimizer = torch.optim.Adam(self.parameters(), lr=self.lr_rate, weight_decay=5e-4)

    lr_dict = {
        "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=5, verbose=True),
        "interval": "epoch",
        "frequency": 1,
        "monitor": "val_acc",
        "strict": True,
        "name": None
        }

    return [optimizer], [lr_dict]

In [74]:
model = myModel(50, 1e-3)
trainer = pl.Trainer(max_epochs = 10)
trainer.fit(model, train_dl, val_dl)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type        | Params
----------------------------------------
0 | model   | MobileNetV2 | 3.5 M 
1 | output  | Linear      | 50.1 K
2 | dropout | Dropout     | 0     
3 | softmax | Softmax     | 0     
----------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.220    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Val_loss: 3.91, Val_acc: 0.00 

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Val_loss: 3.94, Val_acc: 0.00 Train_loss: 3.80, train_acc: 0.14 

Validation: 0it [00:00, ?it/s]

Val_loss: 3.94, Val_acc: 0.00 Train_loss: 3.80, train_acc: 0.14 

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
import pandas as pd

df = pd.read_csv('gt.csv')
df.head

In [17]:
pd.unique(df['class_id'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])